In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from scipy import ndimage
from scipy.signal import find_peaks, savgol_filter
import os
import glob
from google.colab import drive

# Mount Google Drive if using Colab
try:
    drive.mount('/content/drive')
    USING_COLAB = True
except:
    USING_COLAB = False
    print("Not running in Colab, skipping drive mount")

# Define the input and output paths
# Updated to match your directory structure
if USING_COLAB:
    input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x20/Cadherins/background'
    output_tif_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes'
    output_img_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_images'
else:
    # Local paths (change these as needed)
    input_dir = './'
    output_tif_dir = './segmented/tiffs'
    output_img_dir = './segmented/images'

# Create output directories if they don't exist
os.makedirs(output_tif_dir, exist_ok=True)
os.makedirs(output_img_dir, exist_ok=True)

# Find all processed files in the input directory
processed_files = glob.glob(os.path.join(input_dir, "*_hole_projection.tif"))

# If no specific files found, look for any TIF files
if len(processed_files) == 0:
    processed_files = glob.glob(os.path.join(input_dir, "*.tif"))

print(f"Found {len(processed_files)} TIF files to process")
if len(processed_files) == 0:
    print("No TIF files found. Please check the input directory.")
    # Option to look for other file types
    all_files = glob.glob(os.path.join(input_dir, "*"))
    if len(all_files) > 0:
        print(f"Found {len(all_files)} files of other types in the directory.")
        print("Available files:")
        for file in all_files:
            print(f" - {os.path.basename(file)}")

# Function to apply illumination correction
def apply_illumination_correction(img):
    # Normalize image to [0, 1] range for processing
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:  # Check to prevent division by zero
        img_norm = (img - img_min) / (img_max - img_min)
    else:
        img_norm = img.astype(float)
        print("  Warning: Image has constant values, normalization skipped")

    # Estimate background using a large Gaussian filter
    print("Applying illumination correction via division method...")
    sigma = max(img_norm.shape) // 20  # Divisor of 20 for division method
    print(f"  Using Gaussian filter with sigma = {sigma}")

    # Apply Gaussian filter to estimate background
    background = ndimage.gaussian_filter(img_norm, sigma=sigma)

    # Use DIVISION instead of subtraction to correct illumination
    illumination_corrected = img_norm / (background + 1e-6)  # Adding small value to prevent division by zero

    # Normalize the result to [0, 1] range
    illumination_min = np.min(illumination_corrected)
    illumination_max = np.max(illumination_corrected)
    if illumination_max > illumination_min:  # Check to prevent division by zero
        illumination_corrected = (illumination_corrected - illumination_min) / (illumination_max - illumination_min)

    return illumination_corrected

# Function to find threshold using valley-based method
def find_valley_threshold(img):
    # Create a histogram of intensity values
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))

    # Find the peak intensity (mode)
    peak_idx = np.argmax(hist)
    peak_value = bins[peak_idx]
    print(f"Peak found at intensity value: {peak_value:.4f}")

    # Apply a smoothing filter to the histogram to reduce noise
    hist_smooth = savgol_filter(hist, window_length=11, polyorder=3)

    # Find valleys (local minima) in the smoothed histogram
    # We invert the histogram because find_peaks finds maxima
    valleys, _ = find_peaks(-hist_smooth)

    # Find valleys to the left of the peak
    valleys_left = valleys[valleys < peak_idx]

    # If there are valleys to the left, find the closest one to the peak
    if len(valleys_left) > 0:
        first_valley_idx = valleys_left[-1]  # Last valley to the left of the peak
        valley_value = bins[first_valley_idx]
        threshold = valley_value
        print(f"First valley found at intensity value: {valley_value:.4f}")
    else:
        # If no valleys are found, use a fallback method
        # Find where the histogram drops to 50% of the peak height
        half_height = hist[peak_idx] * 0.5
        for i in range(peak_idx, -1, -1):
            if hist[i] < half_height:
                threshold = bins[i]
                print(f"No clear valley found. Using 50% peak height threshold: {threshold:.4f}")
                break
        else:
            threshold = bins[0]
            print("Using minimum value as threshold")

    return threshold, hist, hist_smooth, bins, peak_idx, valleys_left

# Process each file
for file_path in processed_files:
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")

    # Load the image
    try:
        img = tifffile.imread(file_path)
        print(f"Image loaded successfully, shape: {img.shape}, dtype: {img.dtype}")
    except Exception as e:
        print(f"Error loading image {filename}: {e}")
        print("Skipping to next file")
        continue

    # Apply illumination correction
    illumination_corrected = apply_illumination_correction(img)

    # Find threshold using the valley-based method
    threshold, hist, hist_smooth, bins, peak_idx, valleys_left = find_valley_threshold(illumination_corrected)

    # Apply thresholding to segment the image
    segmented = np.zeros_like(illumination_corrected)
    segmented[illumination_corrected <= threshold] = 1  # Segment pixels with intensity <= threshold

    # Visualization
    plt.figure(figsize=(15, 10))

    # Plot the original histogram with the detected peak and valley
    plt.subplot(2, 2, 1)
    plt.bar(bins[:-1], hist, width=bins[1] - bins[0], alpha=0.7)
    plt.plot(bins[:-1], hist_smooth, 'r-', linewidth=2)
    peak_value = bins[peak_idx]
    plt.axvline(x=peak_value, color='g', linestyle='--', label=f'Peak: {peak_value:.4f}')
    plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
    if len(valleys_left) > 0:
        for valley_idx in valleys_left:
            plt.axvline(x=bins[valley_idx], color='y', linestyle=':', alpha=0.5)
    plt.title('Intensity Histogram with Detected Threshold')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')
    plt.legend()

    # Display the original image
    plt.subplot(2, 2, 2)
    plt.imshow(illumination_corrected, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Illumination Corrected Image')

    # Display the segmented image
    plt.subplot(2, 2, 3)
    plt.imshow(segmented, cmap='binary')
    plt.title('Segmented Image')

    # Show a comparison of the original and segmented images
    plt.subplot(2, 2, 4)
    # Create an RGB image for overlay
    overlay = np.zeros((*illumination_corrected.shape, 3))
    # Original in grayscale
    for i in range(3):
        overlay[..., i] = illumination_corrected
    # Segmented areas in red
    overlay[segmented == 1, 0] = 1  # Red channel
    overlay[segmented == 1, 1] = 0  # Green channel
    overlay[segmented == 1, 2] = 0  # Blue channel
    plt.imshow(overlay)
    plt.title('Overlay: Segmented Areas in Red')

    plt.tight_layout()

    # Save the visualization to the images directory
    vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_visualization.png")
    plt.savefig(vis_path, dpi=300)
    print(f"  Saved visualization to: {vis_path}")
    plt.close()

    # Save the illumination corrected image to the TIFF directory
    illum_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_illumination_corrected.tif")
    tifffile.imwrite(illum_path, illumination_corrected.astype(np.float32))
    print(f"  Saved illumination corrected image to: {illum_path}")

    # Save the segmented image to the TIFF directory
    seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented.tif")
    tifffile.imwrite(seg_path, segmented.astype(np.uint8))
    print(f"  Saved segmented image to: {seg_path}")

print("\nAll processing completed!")

Mounted at /content/drive
Found 34 TIF files to process

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional.tif
Image loaded successfully, shape: (1024, 1024), dtype: uint16
Applying illumination correction via division method...
  Using Gaussian filter with sigma = 51
Peak found at intensity value: 0.0312
First valley found at intensity value: 0.0039
  Saved visualization to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_images/denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional_visualization.png
  Saved illumination corrected image to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes/denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional_illumination_corrected.tif
  Saved segmented image to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes/denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional_segmented.tif


with lowered threshold

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from scipy import ndimage
from scipy.signal import find_peaks, savgol_filter
import os
import glob
from google.colab import drive

# Mount Google Drive if using Colab
try:
    drive.mount('/content/drive')
    USING_COLAB = True
except:
    USING_COLAB = False
    print("Not running in Colab, skipping drive mount")

# Define the input and output paths
# Updated to match your directory structure
if USING_COLAB:
    input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x20/Cadherins/background'
    output_tif_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes'
    output_img_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_images'
else:
    # Local paths (change these as needed)
    input_dir = './'
    output_tif_dir = './segmented/tiffs'
    output_img_dir = './segmented/images'

# Create output directories if they don't exist
os.makedirs(output_tif_dir, exist_ok=True)
os.makedirs(output_img_dir, exist_ok=True)

# Find all processed files in the input directory
processed_files = glob.glob(os.path.join(input_dir, "*_hole_projection.tif"))

# If no specific files found, look for any TIF files
if len(processed_files) == 0:
    processed_files = glob.glob(os.path.join(input_dir, "*.tif"))

print(f"Found {len(processed_files)} TIF files to process")
if len(processed_files) == 0:
    print("No TIF files found. Please check the input directory.")
    # Option to look for other file types
    all_files = glob.glob(os.path.join(input_dir, "*"))
    if len(all_files) > 0:
        print(f"Found {len(all_files)} files of other types in the directory.")
        print("Available files:")
        for file in all_files:
            print(f" - {os.path.basename(file)}")

# Function to apply illumination correction
def apply_illumination_correction(img):
    # Normalize image to [0, 1] range for processing
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:  # Check to prevent division by zero
        img_norm = (img - img_min) / (img_max - img_min)
    else:
        img_norm = img.astype(float)
        print("  Warning: Image has constant values, normalization skipped")

    # Estimate background using a large Gaussian filter
    print("Applying illumination correction via division method...")
    sigma = max(img_norm.shape) // 20  # Divisor of 20 for division method
    print(f"  Using Gaussian filter with sigma = {sigma}")

    # Apply Gaussian filter to estimate background
    background = ndimage.gaussian_filter(img_norm, sigma=sigma)

    # Use DIVISION instead of subtraction to correct illumination
    illumination_corrected = img_norm / (background + 1e-6)  # Adding small value to prevent division by zero

    # Normalize the result to [0, 1] range
    illumination_min = np.min(illumination_corrected)
    illumination_max = np.max(illumination_corrected)
    if illumination_max > illumination_min:  # Check to prevent division by zero
        illumination_corrected = (illumination_corrected - illumination_min) / (illumination_max - illumination_min)

    return illumination_corrected

# Function to find threshold using valley-based method
def find_valley_threshold(img):
    # Create a histogram of intensity values
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))

    # Find the peak intensity (mode)
    peak_idx = np.argmax(hist)
    peak_value = bins[peak_idx]
    print(f"Peak found at intensity value: {peak_value:.4f}")

    # Apply a smoothing filter to the histogram to reduce noise
    hist_smooth = savgol_filter(hist, window_length=11, polyorder=3)

    # Find valleys (local minima) in the smoothed histogram
    # We invert the histogram because find_peaks finds maxima
    valleys, _ = find_peaks(-hist_smooth)

    # Find valleys to the left of the peak
    valleys_left = valleys[valleys < peak_idx]

    # If there are valleys to the left, find the closest one to the peak
    if len(valleys_left) > 0:
        first_valley_idx = valleys_left[-1]  # Last valley to the left of the peak
        valley_value = bins[first_valley_idx]
        threshold = valley_value
        print(f"First valley found at intensity value: {threshold:.4f}")
    else:
        # If no valleys are found, use a fallback method
        # Find where the histogram drops to 50% of the peak height
        half_height = hist[peak_idx] * 0.5
        for i in range(peak_idx, -1, -1):
            if hist[i] < half_height:
                threshold = bins[i]
                print(f"No clear valley found. Using 50% peak height threshold: {threshold:.4f}")
                break
        else:
            threshold = bins[0]
            print("Using minimum value as threshold")

    # NEW CODE: Check if threshold is within the valid range for this folder
    # If not, set it to 0 to create a mask without holes
    if threshold < 0.005 or threshold > 0.035:
        print(f"  Threshold {threshold:.4f} is outside the valid range (0.005-0.035) for this folder")
        print("  Setting threshold to 0 to create a mask without holes")
        threshold = 0
    else:
        print(f"  Threshold {threshold:.4f} is within the valid range (0.005-0.035) for this folder")

    return threshold, hist, hist_smooth, bins, peak_idx, valleys_left

# Function to dilate binary mask by specified number of pixels
def dilate_mask(binary_mask, dilation_pixels=3):
    """
    Dilate a binary mask by a specified number of pixels.

    Args:
        binary_mask: Binary mask to dilate (0 and 1 values)
        dilation_pixels: Number of pixels to dilate the mask by

    Returns:
        Dilated binary mask
    """
    # Create a disk-shaped structuring element for dilation
    # The size depends on the number of pixels to dilate
    struct_elem = ndimage.generate_binary_structure(2, 1)  # Basic cross-shaped element

    # Iterate to achieve desired dilation size
    for _ in range(dilation_pixels):
        struct_elem = ndimage.binary_dilation(struct_elem)

    # Perform dilation
    dilated_mask = ndimage.binary_dilation(binary_mask, structure=struct_elem)

    print(f"  Mask dilated by {dilation_pixels} pixels")
    return dilated_mask

# Function to enhance image contrast for better visualization
def enhance_contrast(img, percentile_low=2, percentile_high=98):
    """
    Enhance image contrast using percentile-based contrast stretching.

    Args:
        img: Input image (normalized to [0, 1])
        percentile_low: Lower percentile for contrast stretching (default: 2)
        percentile_high: Upper percentile for contrast stretching (default: 98)

    Returns:
        Contrast-enhanced image
    """
    # Compute percentiles
    p_low = np.percentile(img, percentile_low)
    p_high = np.percentile(img, percentile_high)

    # Avoid division by zero
    if p_high > p_low:
        img_enhanced = np.clip((img - p_low) / (p_high - p_low), 0, 1)
    else:
        img_enhanced = img.copy()
        print("  Warning: Could not enhance contrast (percentiles too close)")

    return img_enhanced

# Process each file
for file_path in processed_files:
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")

    # Load the image
    try:
        img = tifffile.imread(file_path)
        print(f"Image loaded successfully, shape: {img.shape}, dtype: {img.dtype}")
    except Exception as e:
        print(f"Error loading image {filename}: {e}")
        print("Skipping to next file")
        continue

    # Apply illumination correction
    illumination_corrected = apply_illumination_correction(img)

    # Create contrast-enhanced version for visualization
    enhanced_img = enhance_contrast(illumination_corrected, percentile_low=1, percentile_high=99)
    print(f"  Applied contrast enhancement for visualization")

    # Find threshold using the valley-based method
    threshold, hist, hist_smooth, bins, peak_idx, valleys_left = find_valley_threshold(illumination_corrected)

    # Apply thresholding to segment the image
    segmented = np.zeros_like(illumination_corrected)
    if threshold > 0:  # Only apply thresholding if the threshold is greater than 0
        segmented[illumination_corrected <= threshold] = 1  # Segment pixels with intensity <= threshold
        print(f"  Created mask with holes based on threshold: {threshold:.4f}")
    else:
        # Empty mask (no holes)
        print("  Created empty mask (no holes) as threshold is outside valid range")

    # Dilate the segmented mask by 3 pixels
    dilated_segmented = dilate_mask(segmented, dilation_pixels=3)

    # Visualization (now includes both original and dilated segmentation)
    plt.figure(figsize=(16, 20))

    # Plot the original histogram with the detected peak and valley
    plt.subplot(4, 2, 1)
    plt.bar(bins[:-1], hist, width=bins[1] - bins[0], alpha=0.7)
    plt.plot(bins[:-1], hist_smooth, 'r-', linewidth=2)
    peak_value = bins[peak_idx]
    plt.axvline(x=peak_value, color='g', linestyle='--', label=f'Peak: {peak_value:.4f}')

    # Add valid range indication
    plt.axvspan(0.005, 0.035, alpha=0.2, color='green', label='Valid range (0.005-0.035)')

    if threshold > 0:
        plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
    else:
        plt.axvline(x=0, color='r', linestyle='--', label='Threshold: 0 (outside valid range)')

    if len(valleys_left) > 0:
        for valley_idx in valleys_left:
            valley_value = bins[valley_idx]
            plt.axvline(x=valley_value, color='y', linestyle=':', alpha=0.5)

    plt.title('Intensity Histogram with Detected Threshold')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')
    plt.legend()

    # Display the illumination corrected image (original)
    plt.subplot(4, 2, 2)
    plt.imshow(illumination_corrected, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Illumination Corrected Image (Original)')

    # Display the enhanced image for better visualization
    plt.subplot(4, 2, 3)
    plt.imshow(enhanced_img, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Contrast-Enhanced Image (For Visualization)')

    # Display the original segmented image
    plt.subplot(4, 2, 4)
    plt.imshow(segmented, cmap='binary')
    if threshold > 0:
        plt.title(f'Original Segmented Image (Threshold: {threshold:.4f})')
    else:
        plt.title('Original Segmented Image (Empty - Threshold outside range)')

    # Display the dilated segmented image
    plt.subplot(4, 2, 5)
    plt.imshow(dilated_segmented, cmap='binary')
    if threshold > 0:
        plt.title(f'Dilated Segmented Image (3 pixels, Threshold: {threshold:.4f})')
    else:
        plt.title('Dilated Segmented Image (Empty - Threshold outside range)')

    # Show overlay with original segmentation over enhanced image
    plt.subplot(4, 2, 6)
    # Create an RGB image for overlay
    overlay_orig = np.zeros((*enhanced_img.shape, 3))
    # Enhanced image in grayscale
    for i in range(3):
        overlay_orig[..., i] = enhanced_img
    # Segmented areas in red (more vibrant)
    overlay_orig[segmented == 1, 0] = 1.0    # Red channel - maximum
    overlay_orig[segmented == 1, 1] = 0.0    # Green channel - minimum
    overlay_orig[segmented == 1, 2] = 0.0    # Blue channel - minimum
    plt.imshow(overlay_orig)
    if threshold > 0:
        plt.title(f'Overlay: Original Segmentation (Red, Threshold: {threshold:.4f})')
    else:
        plt.title('Overlay: Original Segmentation (Empty - Threshold outside range)')

    # Show overlay with dilated segmentation over enhanced image
    plt.subplot(4, 2, 7)
    # Create an RGB image for overlay
    overlay_dilated = np.zeros((*enhanced_img.shape, 3))
    # Enhanced image in grayscale
    for i in range(3):
        overlay_dilated[..., i] = enhanced_img
    # Dilated segmented areas in red
    overlay_dilated[dilated_segmented == 1, 0] = 1.0  # Red channel
    overlay_dilated[dilated_segmented == 1, 1] = 0.0  # Green channel
    overlay_dilated[dilated_segmented == 1, 2] = 0.0  # Blue channel
    plt.imshow(overlay_dilated)
    if threshold > 0:
        plt.title(f'Overlay: Dilated Segmentation (Red, Threshold: {threshold:.4f})')
    else:
        plt.title('Overlay: Dilated Segmentation (Empty - Threshold outside range)')

    # Create a contour overlay for clearer visualization
    plt.subplot(4, 2, 8)
    # Erode the dilated mask to get the contour
    eroded = ndimage.binary_erosion(dilated_segmented)
    contour = dilated_segmented.astype(np.float32) - eroded.astype(np.float32)

    # Create the contour overlay
    contour_overlay = np.zeros((*enhanced_img.shape, 3))
    # Enhanced image in grayscale
    for i in range(3):
        contour_overlay[..., i] = enhanced_img
    # Contours in bright green for high visibility
    contour_overlay[contour > 0, 0] = 0.0     # Red channel - off
    contour_overlay[contour > 0, 1] = 1.0     # Green channel - maximum
    contour_overlay[contour > 0, 2] = 0.0     # Blue channel - off

    # Fill the regions with semi-transparent red
    masked_regions = dilated_segmented > 0
    contour_overlay[masked_regions, 0] = np.maximum(contour_overlay[masked_regions, 0], 0.5)  # Redder
    contour_overlay[masked_regions, 1] *= 0.7  # Reduce green in masked regions
    contour_overlay[masked_regions, 2] *= 0.7  # Reduce blue in masked regions

    plt.imshow(contour_overlay)
    if threshold > 0:
        plt.title(f'Contour Overlay: Dilated Segmentation (Green outline, Threshold: {threshold:.4f})')
    else:
        plt.title('Contour Overlay: Empty (Threshold outside range)')

    plt.tight_layout()

    # Save the visualization to the images directory
    vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_visualization.png")
    plt.savefig(vis_path, dpi=300)
    print(f"  Saved visualization to: {vis_path}")
    plt.close()

    # Create a simplified visualization for quick reference
    plt.figure(figsize=(12, 5))

    # Left - contrast enhanced image
    plt.subplot(1, 2, 1)
    plt.imshow(enhanced_img, cmap='gray')
    plt.title('Contrast-Enhanced Image')
    plt.axis('off')

    # Right - high contrast contour overlay
    plt.subplot(1, 2, 2)
    plt.imshow(contour_overlay)
    if threshold > 0:
        plt.title(f'Segmented Holes with Contours (Threshold: {threshold:.4f})')
    else:
        plt.title('No Holes Detected (Threshold outside valid range)')
    plt.axis('off')

    plt.tight_layout()

    # Save the simplified visualization
    simple_vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_summary.png")
    plt.savefig(simple_vis_path, dpi=300)
    print(f"  Saved simplified visualization to: {simple_vis_path}")
    plt.close()

    # Save the illumination corrected image to the TIFF directory
    illum_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_illumination_corrected.tif")
    tifffile.imwrite(illum_path, illumination_corrected.astype(np.float32))
    print(f"  Saved illumination corrected image to: {illum_path}")

    # Save the contrast-enhanced image to the TIFF directory
    enhanced_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_enhanced.tif")
    tifffile.imwrite(enhanced_path, enhanced_img.astype(np.float32))
    print(f"  Saved contrast-enhanced image to: {enhanced_path}")

    # Save the original segmented image to the TIFF directory
    seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented.tif")
    tifffile.imwrite(seg_path, segmented.astype(np.uint8))
    if threshold > 0:
        print(f"  Saved original segmented image to: {seg_path} (with holes)")
    else:
        print(f"  Saved original segmented image to: {seg_path} (without holes)")

    # Save the dilated segmented image to the TIFF directory
    dilated_seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented_dilated.tif")
    tifffile.imwrite(dilated_seg_path, dilated_segmented.astype(np.uint8))
    if threshold > 0:
        print(f"  Saved dilated segmented image to: {dilated_seg_path} (with holes)")
    else:
        print(f"  Saved dilated segmented image to: {dilated_seg_path} (without holes)")

print("\nAll processing completed!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 23 TIF files to process

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional.tif
Image loaded successfully, shape: (1024, 1024), dtype: uint16
Applying illumination correction via division method...
  Using Gaussian filter with sigma = 51
  Applied contrast enhancement for visualization
Peak found at intensity value: 0.0312
First valley found at intensity value: 0.0039
  Threshold 0.0039 is outside the valid range (0.005-0.035) for this folder
  Setting threshold to 0 to create a mask without holes
  Created empty mask (no holes) as threshold is outside valid range
  Mask dilated by 3 pixels
  Saved visualization to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_images/denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional_visualization.png
  Saved simplified visualization to

Manual check and removal

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from scipy import ndimage
from scipy.signal import find_peaks, savgol_filter
import os
import glob
from google.colab import drive

# Mount Google Drive if using Colab
try:
    drive.mount('/content/drive')
    USING_COLAB = True
except:
    USING_COLAB = False
    print("Not running in Colab, skipping drive mount")

# Define the input and output paths
# Updated to match your directory structure
if USING_COLAB:
    input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x20/Cadherins/background'
    output_tif_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes'
    output_img_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_images'
else:
    # Local paths (change these as needed)
    input_dir = './'
    output_tif_dir = './segmented/tiffs'
    output_img_dir = './segmented/images'

# Create output directories if they don't exist
os.makedirs(output_tif_dir, exist_ok=True)
os.makedirs(output_img_dir, exist_ok=True)

# Target file name to process - 16
TARGET_FILE = "denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016_Cadherins_regional"

# Find the target file in the input directory
target_files = glob.glob(os.path.join(input_dir, f"{TARGET_FILE}*.tif"))

if len(target_files) == 0:
    print(f"Target file '{TARGET_FILE}*.tif' not found. Please check the input directory.")
    # Look for any files that might match partially
    similar_files = glob.glob(os.path.join(input_dir, f"*{TARGET_FILE.split('_')[0]}*.tif"))
    if len(similar_files) > 0:
        print(f"Found {len(similar_files)} files with similar names:")
        for file in similar_files:
            print(f" - {os.path.basename(file)}")

    # Display all available files
    all_files = glob.glob(os.path.join(input_dir, "*.tif"))
    if len(all_files) > 0:
        print(f"\nAll available TIF files ({len(all_files)}):")
        for file in all_files[:10]:  # Show only first 10 to avoid clutter
            print(f" - {os.path.basename(file)}")
        if len(all_files) > 10:
            print(f" ... and {len(all_files) - 10} more")
else:
    print(f"Found {len(target_files)} matching file(s):")
    for file in target_files:
        print(f" - {os.path.basename(file)}")

# Function to apply illumination correction
def apply_illumination_correction(img):
    # Normalize image to [0, 1] range for processing
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:  # Check to prevent division by zero
        img_norm = (img - img_min) / (img_max - img_min)
    else:
        img_norm = img.astype(float)
        print("  Warning: Image has constant values, normalization skipped")

    # Estimate background using a large Gaussian filter
    print("Applying illumination correction via division method...")
    sigma = max(img_norm.shape) // 20  # Divisor of 20 for division method
    print(f"  Using Gaussian filter with sigma = {sigma}")

    # Apply Gaussian filter to estimate background
    background = ndimage.gaussian_filter(img_norm, sigma=sigma)

    # Use DIVISION instead of subtraction to correct illumination
    illumination_corrected = img_norm / (background + 1e-6)  # Adding small value to prevent division by zero

    # Normalize the result to [0, 1] range
    illumination_min = np.min(illumination_corrected)
    illumination_max = np.max(illumination_corrected)
    if illumination_max > illumination_min:  # Check to prevent division by zero
        illumination_corrected = (illumination_corrected - illumination_min) / (illumination_max - illumination_min)

    return illumination_corrected

# Function to find threshold using valley-based method
def find_valley_threshold(img):
    # Create a histogram of intensity values
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))

    # Find the peak intensity (mode)
    peak_idx = np.argmax(hist)
    peak_value = bins[peak_idx]
    print(f"Peak found at intensity value: {peak_value:.4f}")

    # Apply a smoothing filter to the histogram to reduce noise
    hist_smooth = savgol_filter(hist, window_length=11, polyorder=3)

    # Find valleys (local minima) in the smoothed histogram
    # We invert the histogram because find_peaks finds maxima
    valleys, _ = find_peaks(-hist_smooth)

    # Find valleys to the left of the peak
    valleys_left = valleys[valleys < peak_idx]

    # If there are valleys to the left, find the closest one to the peak
    if len(valleys_left) > 0:
        first_valley_idx = valleys_left[-1]  # Last valley to the left of the peak
        valley_value = bins[first_valley_idx]
        threshold = valley_value
        print(f"First valley found at intensity value: {threshold:.4f}")
    else:
        # If no valleys are found, use a fallback method
        # Find where the histogram drops to 50% of the peak height
        half_height = hist[peak_idx] * 0.5
        for i in range(peak_idx, -1, -1):
            if hist[i] < half_height:
                threshold = bins[i]
                print(f"No clear valley found. Using 50% peak height threshold: {threshold:.4f}")
                break
        else:
            threshold = bins[0]
            print("Using minimum value as threshold")

    return threshold, hist, hist_smooth, bins, peak_idx, valleys_left

# Function to dilate binary mask by specified number of pixels
def dilate_mask(binary_mask, dilation_pixels=3):
    # Create a disk-shaped structuring element for dilation
    struct_elem = ndimage.generate_binary_structure(2, 1)  # Basic cross-shaped element

    # Iterate to achieve desired dilation size
    for _ in range(dilation_pixels):
        struct_elem = ndimage.binary_dilation(struct_elem)

    # Perform dilation
    dilated_mask = ndimage.binary_dilation(binary_mask, structure=struct_elem)

    print(f"  Mask dilated by {dilation_pixels} pixels")
    return dilated_mask

# Function to enhance image contrast for better visualization
def enhance_contrast(img, percentile_low=2, percentile_high=98):
    # Compute percentiles
    p_low = np.percentile(img, percentile_low)
    p_high = np.percentile(img, percentile_high)

    # Avoid division by zero
    if p_high > p_low:
        img_enhanced = np.clip((img - p_low) / (p_high - p_low), 0, 1)
    else:
        img_enhanced = img.copy()
        print("  Warning: Could not enhance contrast (percentiles too close)")

    return img_enhanced

# Process each file
for file_path in target_files:
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")

    # Load the image
    try:
        img = tifffile.imread(file_path)
        print(f"Image loaded successfully, shape: {img.shape}, dtype: {img.dtype}")
    except Exception as e:
        print(f"Error loading image {filename}: {e}")
        print("Skipping to next file")
        continue

    # Apply illumination correction
    illumination_corrected = apply_illumination_correction(img)

    # Create contrast-enhanced version for visualization
    enhanced_img = enhance_contrast(illumination_corrected, percentile_low=1, percentile_high=99)
    print(f"  Applied contrast enhancement for visualization")

    # Find threshold using the valley-based method
    # This gets the automatically detected threshold for information purposes
    auto_threshold, hist, hist_smooth, bins, peak_idx, valleys_left = find_valley_threshold(illumination_corrected)

    # Special case for this file - use a manual threshold instead
    # We're setting this to 0.014 based on the image analysis
    threshold = 0.014
    print(f"  Using manual threshold override for this specific file: {threshold:.4f}")
    print(f"  (Auto-detected threshold was: {auto_threshold:.4f})")

    # Apply thresholding to segment the image
    segmented = np.zeros_like(illumination_corrected)
    segmented[illumination_corrected <= threshold] = 1  # Segment pixels with intensity <= threshold
    print(f"  Created mask with holes based on manual threshold: {threshold:.4f}")

    # Dilate the segmented mask by 3 pixels
    dilated_segmented = dilate_mask(segmented, dilation_pixels=3)

    # Visualization (now includes both original and dilated segmentation)
    plt.figure(figsize=(16, 20))

    # Plot the original histogram with the detected peak and valley
    plt.subplot(4, 2, 1)
    plt.bar(bins[:-1], hist, width=bins[1] - bins[0], alpha=0.7)
    plt.plot(bins[:-1], hist_smooth, 'r-', linewidth=2)
    peak_value = bins[peak_idx]
    plt.axvline(x=peak_value, color='g', linestyle='--', label=f'Peak: {peak_value:.4f}')

    # Add valid range indication
    plt.axvspan(0.005, 0.035, alpha=0.2, color='green', label='Regular valid range (0.005-0.035)')

    # Show both the auto-threshold and manual threshold
    plt.axvline(x=auto_threshold, color='y', linestyle='--', alpha=0.5, label=f'Auto-threshold: {auto_threshold:.4f}')
    plt.axvline(x=threshold, color='r', linestyle='--', label=f'Manual threshold: {threshold:.4f}')

    if len(valleys_left) > 0:
        for valley_idx in valleys_left:
            valley_value = bins[valley_idx]
            plt.axvline(x=valley_value, color='y', linestyle=':', alpha=0.3)

    plt.title('Intensity Histogram with Manual Threshold')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')
    plt.legend()

    # Display the illumination corrected image (original)
    plt.subplot(4, 2, 2)
    plt.imshow(illumination_corrected, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Illumination Corrected Image (Original)')

    # Display the enhanced image for better visualization
    plt.subplot(4, 2, 3)
    plt.imshow(enhanced_img, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Contrast-Enhanced Image (For Visualization)')

    # Display the original segmented image
    plt.subplot(4, 2, 4)
    plt.imshow(segmented, cmap='binary')
    plt.title(f'Original Segmented Image (Manual Threshold: {threshold:.4f})')

    # Display the dilated segmented image
    plt.subplot(4, 2, 5)
    plt.imshow(dilated_segmented, cmap='binary')
    plt.title(f'Dilated Segmented Image (3 pixels, Manual Threshold: {threshold:.4f})')

    # Show overlay with original segmentation over enhanced image
    plt.subplot(4, 2, 6)
    # Create an RGB image for overlay
    overlay_orig = np.zeros((*enhanced_img.shape, 3))
    # Enhanced image in grayscale
    for i in range(3):
        overlay_orig[..., i] = enhanced_img
    # Segmented areas in red (more vibrant)
    overlay_orig[segmented == 1, 0] = 1.0    # Red channel - maximum
    overlay_orig[segmented == 1, 1] = 0.0    # Green channel - minimum
    overlay_orig[segmented == 1, 2] = 0.0    # Blue channel - minimum
    plt.imshow(overlay_orig)
    plt.title(f'Overlay: Original Segmentation (Red, Manual Threshold: {threshold:.4f})')

    # Show overlay with dilated segmentation over enhanced image
    plt.subplot(4, 2, 7)
    # Create an RGB image for overlay
    overlay_dilated = np.zeros((*enhanced_img.shape, 3))
    # Enhanced image in grayscale
    for i in range(3):
        overlay_dilated[..., i] = enhanced_img
    # Dilated segmented areas in red
    overlay_dilated[dilated_segmented == 1, 0] = 1.0  # Red channel
    overlay_dilated[dilated_segmented == 1, 1] = 0.0  # Green channel
    overlay_dilated[dilated_segmented == 1, 2] = 0.0  # Blue channel
    plt.imshow(overlay_dilated)
    plt.title(f'Overlay: Dilated Segmentation (Red, Manual Threshold: {threshold:.4f})')

    # Create a contour overlay for clearer visualization
    plt.subplot(4, 2, 8)
    # Erode the dilated mask to get the contour
    eroded = ndimage.binary_erosion(dilated_segmented)
    contour = dilated_segmented.astype(np.float32) - eroded.astype(np.float32)

    # Create the contour overlay
    contour_overlay = np.zeros((*enhanced_img.shape, 3))
    # Enhanced image in grayscale
    for i in range(3):
        contour_overlay[..., i] = enhanced_img
    # Contours in bright green for high visibility
    contour_overlay[contour > 0, 0] = 0.0     # Red channel - off
    contour_overlay[contour > 0, 1] = 1.0     # Green channel - maximum
    contour_overlay[contour > 0, 2] = 0.0     # Blue channel - off

    # Fill the regions with semi-transparent red
    masked_regions = dilated_segmented > 0
    contour_overlay[masked_regions, 0] = np.maximum(contour_overlay[masked_regions, 0], 0.5)  # Redder
    contour_overlay[masked_regions, 1] *= 0.7  # Reduce green in masked regions
    contour_overlay[masked_regions, 2] *= 0.7  # Reduce blue in masked regions

    plt.imshow(contour_overlay)
    plt.title(f'Contour Overlay: Dilated Segmentation (Green outline)')

    plt.tight_layout()

    # Save the visualization to the images directory - using original filename (overwriting)
    vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_visualization.png")
    plt.savefig(vis_path, dpi=300)
    print(f"  Saved visualization to: {vis_path} (OVERWRITING ORIGINAL)")
    plt.close()

    # Create a simplified visualization for quick reference
    plt.figure(figsize=(12, 5))

    # Left - contrast enhanced image
    plt.subplot(1, 2, 1)
    plt.imshow(enhanced_img, cmap='gray')
    plt.title('Contrast-Enhanced Image')
    plt.axis('off')

    # Right - high contrast contour overlay
    plt.subplot(1, 2, 2)
    plt.imshow(contour_overlay)
    plt.title(f'Segmented Holes with Contours (Manual Threshold: {threshold:.4f})')
    plt.axis('off')

    plt.tight_layout()

    # Save the simplified visualization - using original filename (overwriting)
    simple_vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_summary.png")
    plt.savefig(simple_vis_path, dpi=300)
    print(f"  Saved simplified visualization to: {simple_vis_path} (OVERWRITING ORIGINAL)")
    plt.close()

    # Save the illumination corrected image - using original filename (overwriting)
    illum_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_illumination_corrected.tif")
    tifffile.imwrite(illum_path, illumination_corrected.astype(np.float32))
    print(f"  Saved illumination corrected image to: {illum_path} (OVERWRITING ORIGINAL)")

    # Save the contrast-enhanced image - using original filename (overwriting)
    enhanced_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_enhanced.tif")
    tifffile.imwrite(enhanced_path, enhanced_img.astype(np.float32))
    print(f"  Saved contrast-enhanced image to: {enhanced_path} (OVERWRITING ORIGINAL)")

    # Save the original segmented image - using original filename (overwriting)
    seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented.tif")
    tifffile.imwrite(seg_path, segmented.astype(np.uint8))
    print(f"  Saved original segmented image to: {seg_path} (OVERWRITING ORIGINAL)")

    # Save the dilated segmented image - using original filename (overwriting)
    dilated_seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented_dilated.tif")
    tifffile.imwrite(dilated_seg_path, dilated_segmented.astype(np.uint8))
    print(f"  Saved dilated segmented image to: {dilated_seg_path} (OVERWRITING ORIGINAL)")

print("\nProcessing completed with original files overwritten!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1 matching file(s):
 - denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016_Cadherins_regional.tif

Processing: denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016_Cadherins_regional.tif
Image loaded successfully, shape: (1024, 1024), dtype: uint16
Applying illumination correction via division method...
  Using Gaussian filter with sigma = 51
  Applied contrast enhancement for visualization
Peak found at intensity value: 0.0312
First valley found at intensity value: 0.0039
  Using manual threshold override for this specific file: 0.0140
  (Auto-detected threshold was: 0.0039)
  Created mask with holes based on manual threshold: 0.0140
  Mask dilated by 3 pixels
  Saved visualization to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_images/denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016_Cadherins_regional_visualization.png

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from scipy import ndimage
from scipy.signal import find_peaks, savgol_filter
import os
import glob
from google.colab import drive

# Mount Google Drive if using Colab
try:
    drive.mount('/content/drive')
    USING_COLAB = True
except:
    USING_COLAB = False
    print("Not running in Colab, skipping drive mount")

# Define the input and output paths
# Updated to match your directory structure
if USING_COLAB:
    input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x20/Cadherins/background'
    output_tif_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes'
    output_img_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_images'
else:
    # Local paths (change these as needed)
    input_dir = './'
    output_tif_dir = './segmented/tiffs'
    output_img_dir = './segmented/images'

# Create output directories if they don't exist
os.makedirs(output_tif_dir, exist_ok=True)
os.makedirs(output_img_dir, exist_ok=True)

# Target files that should have void masks (no holes)
TARGET_FILES = [
    "denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq011_Cadherins_regional",
    "denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_Cadherins_regional"
]

# Find the target files in the input directory
target_files = []
for target in TARGET_FILES:
    files = glob.glob(os.path.join(input_dir, f"{target}*.tif"))
    target_files.extend(files)

if len(target_files) == 0:
    print(f"No target files found. Please check the input directory.")
    # Look for any files that might match partially
    similar_files = []
    for target in TARGET_FILES:
        base_name = target.split('_')[0]
        similar = glob.glob(os.path.join(input_dir, f"*{base_name}*.tif"))
        similar_files.extend(similar)

    if len(similar_files) > 0:
        print(f"Found {len(similar_files)} files with similar names:")
        for file in similar_files:
            print(f" - {os.path.basename(file)}")

    # Display all available files
    all_files = glob.glob(os.path.join(input_dir, "*.tif"))
    if len(all_files) > 0:
        print(f"\nAll available TIF files ({len(all_files)}):")
        for file in all_files[:10]:  # Show only first 10 to avoid clutter
            print(f" - {os.path.basename(file)}")
        if len(all_files) > 10:
            print(f" ... and {len(all_files) - 10} more")
else:
    print(f"Found {len(target_files)} matching file(s):")
    for file in target_files:
        print(f" - {os.path.basename(file)}")

# Function to apply illumination correction
def apply_illumination_correction(img):
    # Normalize image to [0, 1] range for processing
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:  # Check to prevent division by zero
        img_norm = (img - img_min) / (img_max - img_min)
    else:
        img_norm = img.astype(float)
        print("  Warning: Image has constant values, normalization skipped")

    # Estimate background using a large Gaussian filter
    print("Applying illumination correction via division method...")
    sigma = max(img_norm.shape) // 20  # Divisor of 20 for division method
    print(f"  Using Gaussian filter with sigma = {sigma}")

    # Apply Gaussian filter to estimate background
    background = ndimage.gaussian_filter(img_norm, sigma=sigma)

    # Use DIVISION instead of subtraction to correct illumination
    illumination_corrected = img_norm / (background + 1e-6)  # Adding small value to prevent division by zero

    # Normalize the result to [0, 1] range
    illumination_min = np.min(illumination_corrected)
    illumination_max = np.max(illumination_corrected)
    if illumination_max > illumination_min:  # Check to prevent division by zero
        illumination_corrected = (illumination_corrected - illumination_min) / (illumination_max - illumination_min)

    return illumination_corrected

# Function to find threshold using valley-based method
def find_valley_threshold(img):
    # Create a histogram of intensity values
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))

    # Find the peak intensity (mode)
    peak_idx = np.argmax(hist)
    peak_value = bins[peak_idx]
    print(f"Peak found at intensity value: {peak_value:.4f}")

    # Apply a smoothing filter to the histogram to reduce noise
    hist_smooth = savgol_filter(hist, window_length=11, polyorder=3)

    # Find valleys (local minima) in the smoothed histogram
    # We invert the histogram because find_peaks finds maxima
    valleys, _ = find_peaks(-hist_smooth)

    # Find valleys to the left of the peak
    valleys_left = valleys[valleys < peak_idx]

    # If there are valleys to the left, find the closest one to the peak
    if len(valleys_left) > 0:
        first_valley_idx = valleys_left[-1]  # Last valley to the left of the peak
        valley_value = bins[first_valley_idx]
        threshold = valley_value
        print(f"First valley found at intensity value: {threshold:.4f}")
    else:
        # If no valleys are found, use a fallback method
        # Find where the histogram drops to 50% of the peak height
        half_height = hist[peak_idx] * 0.5
        for i in range(peak_idx, -1, -1):
            if hist[i] < half_height:
                threshold = bins[i]
                print(f"No clear valley found. Using 50% peak height threshold: {threshold:.4f}")
                break
        else:
            threshold = bins[0]
            print("Using minimum value as threshold")

    return threshold, hist, hist_smooth, bins, peak_idx, valleys_left

# Function to enhance image contrast for better visualization
def enhance_contrast(img, percentile_low=2, percentile_high=98):
    # Compute percentiles
    p_low = np.percentile(img, percentile_low)
    p_high = np.percentile(img, percentile_high)

    # Avoid division by zero
    if p_high > p_low:
        img_enhanced = np.clip((img - p_low) / (p_high - p_low), 0, 1)
    else:
        img_enhanced = img.copy()
        print("  Warning: Could not enhance contrast (percentiles too close)")

    return img_enhanced

# Process each file
for file_path in target_files:
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")

    # Load the image
    try:
        img = tifffile.imread(file_path)
        print(f"Image loaded successfully, shape: {img.shape}, dtype: {img.dtype}")
    except Exception as e:
        print(f"Error loading image {filename}: {e}")
        print("Skipping to next file")
        continue

    # Apply illumination correction
    illumination_corrected = apply_illumination_correction(img)

    # Create contrast-enhanced version for visualization
    enhanced_img = enhance_contrast(illumination_corrected, percentile_low=1, percentile_high=99)
    print(f"  Applied contrast enhancement for visualization")

    # Find threshold using the valley-based method
    # This gets the automatically detected threshold for information purposes
    auto_threshold, hist, hist_smooth, bins, peak_idx, valleys_left = find_valley_threshold(illumination_corrected)

    # Special case for these files - force a void mask (no holes)
    # We'll set the threshold to 0, which means no pixels will be segmented
    threshold = 0
    print(f"  Creating void mask for this file (no holes)")
    print(f"  (Auto-detected threshold was: {auto_threshold:.4f}, but is being ignored)")

    # Create an empty segmentation (no holes)
    segmented = np.zeros_like(illumination_corrected)
    print(f"  Created void mask (no holes) as per requirements for this file")

    # The dilated segmentation will also be empty
    dilated_segmented = np.zeros_like(illumination_corrected)
    print(f"  Dilated mask is also empty (no holes)")

    # Visualization
    plt.figure(figsize=(16, 20))

    # Plot the original histogram with the detected peak
    plt.subplot(4, 2, 1)
    plt.bar(bins[:-1], hist, width=bins[1] - bins[0], alpha=0.7)
    plt.plot(bins[:-1], hist_smooth, 'r-', linewidth=2)
    peak_value = bins[peak_idx]
    plt.axvline(x=peak_value, color='g', linestyle='--', label=f'Peak: {peak_value:.4f}')

    # Add valid range indication
    plt.axvspan(0.005, 0.035, alpha=0.2, color='green', label='Regular valid range (0.005-0.035)')

    # Show the auto-threshold for reference
    plt.axvline(x=auto_threshold, color='y', linestyle='--', alpha=0.5, label=f'Auto-threshold: {auto_threshold:.4f}')

    if len(valleys_left) > 0:
        for valley_idx in valleys_left:
            valley_value = bins[valley_idx]
            plt.axvline(x=valley_value, color='y', linestyle=':', alpha=0.3)

    plt.title('Intensity Histogram (Void Mask Applied)')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')
    plt.legend()

    # Display the illumination corrected image (original)
    plt.subplot(4, 2, 2)
    plt.imshow(illumination_corrected, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Illumination Corrected Image (Original)')

    # Display the enhanced image for better visualization
    plt.subplot(4, 2, 3)
    plt.imshow(enhanced_img, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Contrast-Enhanced Image (For Visualization)')

    # Display the original segmented image (empty)
    plt.subplot(4, 2, 4)
    plt.imshow(segmented, cmap='binary')
    plt.title('Original Segmented Image (Void Mask - No Holes)')

    # Display the dilated segmented image (empty)
    plt.subplot(4, 2, 5)
    plt.imshow(dilated_segmented, cmap='binary')
    plt.title('Dilated Segmented Image (Void Mask - No Holes)')

    # Show overlay with original segmentation over enhanced image (just the enhanced image)
    plt.subplot(4, 2, 6)
    plt.imshow(enhanced_img, cmap='gray')
    plt.title('Enhanced Image (No Segmentation Overlay)')

    # Show overlay with dilated segmentation over enhanced image (just the enhanced image)
    plt.subplot(4, 2, 7)
    plt.imshow(enhanced_img, cmap='gray')
    plt.title('Enhanced Image (No Segmentation Overlay)')

    # Create a notice for the user about the void mask
    plt.subplot(4, 2, 8)
    # Create a white background
    placeholder = np.ones_like(enhanced_img)
    plt.imshow(placeholder, cmap='gray', vmin=0, vmax=1)
    plt.text(0.5, 0.5, 'VOID MASK\nNo holes detected\nin this image',
             horizontalalignment='center', verticalalignment='center',
             transform=plt.gca().transAxes, fontsize=20, color='black')
    plt.axis('off')
    plt.title('Special Processing Note')

    plt.tight_layout()

    # Save the visualization to the images directory - using original naming convention
    vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_visualization.png")
    plt.savefig(vis_path, dpi=300)
    print(f"  Saved visualization to: {vis_path}")
    plt.close()

    # Create a simplified visualization for quick reference
    plt.figure(figsize=(12, 5))

    # Left - contrast enhanced image
    plt.subplot(1, 2, 1)
    plt.imshow(enhanced_img, cmap='gray')
    plt.title('Contrast-Enhanced Image')
    plt.axis('off')

    # Right - notification of void mask
    plt.subplot(1, 2, 2)
    # White background
    placeholder = np.ones_like(enhanced_img)
    plt.imshow(placeholder, cmap='gray', vmin=0, vmax=1)
    plt.text(0.5, 0.5, 'VOID MASK\nNo holes detected\nin this image',
             horizontalalignment='center', verticalalignment='center',
             transform=plt.gca().transAxes, fontsize=16, color='black')
    plt.title('No Segmentation Applied')
    plt.axis('off')

    plt.tight_layout()

    # Save the simplified visualization - using original naming convention
    simple_vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_summary.png")
    plt.savefig(simple_vis_path, dpi=300)
    print(f"  Saved simplified visualization to: {simple_vis_path}")
    plt.close()

    # Save the illumination corrected image - using original naming convention
    illum_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_illumination_corrected.tif")
    tifffile.imwrite(illum_path, illumination_corrected.astype(np.float32))
    print(f"  Saved illumination corrected image to: {illum_path}")

    # Save the contrast-enhanced image - using original naming convention
    enhanced_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_enhanced.tif")
    tifffile.imwrite(enhanced_path, enhanced_img.astype(np.float32))
    print(f"  Saved contrast-enhanced image to: {enhanced_path}")

    # Save the empty segmented image - using original naming convention (OVERWRITING)
    seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented.tif")
    tifffile.imwrite(seg_path, segmented.astype(np.uint8))
    print(f"  Saved void segmented image to: {seg_path} (no holes) - OVERWRITING ORIGINAL")

    # Save the empty dilated segmented image - using original naming convention (OVERWRITING)
    dilated_seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented_dilated.tif")
    tifffile.imwrite(dilated_seg_path, dilated_segmented.astype(np.uint8))
    print(f"  Saved void dilated segmented image to: {dilated_seg_path} (no holes) - OVERWRITING ORIGINAL")

print("\nProcessing completed with original segmentation masks overwritten!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 2 matching file(s):
 - denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq011_Cadherins_regional.tif
 - denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_Cadherins_regional.tif

Processing: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq011_Cadherins_regional.tif
Image loaded successfully, shape: (1024, 1024), dtype: uint16
Applying illumination correction via division method...
  Using Gaussian filter with sigma = 51
  Applied contrast enhancement for visualization
Peak found at intensity value: 0.0234
No clear valley found. Using 50% peak height threshold: 0.0117
  Creating void mask for this file (no holes)
  (Auto-detected threshold was: 0.0117, but is being ignored)
  Created void mask (no holes) as per requirements for this file
  Dilated mask is also empty (no holes)
  Saved visualization to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segme

Saving masks

In [17]:
import os
import glob
import shutil
from google.colab import drive

# Mount Google Drive if using Colab
try:
    drive.mount('/content/drive')
    USING_COLAB = True
except:
    USING_COLAB = False
    print("Not running in Colab, skipping drive mount")

# Define the paths
if USING_COLAB:
    source_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes'
    target_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_masks'
else:
    # Local paths (change these as needed)
    source_dir = './segmented/tiffs'
    target_dir = './segmented/masks'

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)
print(f"Target directory for masks: {target_dir}")

# Find all dilated segmented mask files in the source directory
mask_files = glob.glob(os.path.join(source_dir, "*_segmented_dilated.tif"))

if len(mask_files) == 0:
    print("No mask files found in the source directory.")
else:
    print(f"Found {len(mask_files)} mask files to move:")

    # Move each mask file to the target directory
    for file_path in mask_files:
        filename = os.path.basename(file_path)
        destination = os.path.join(target_dir, filename)

        # Option 1: Copy the file (keeping original in place)
        shutil.copy2(file_path, destination)
        print(f"  Copied: {filename} to {target_dir}")

        # Option 2: Move the file (uncomment if you want to move instead of copy)
        # shutil.move(file_path, destination)
        # print(f"  Moved: {filename} to {target_dir}")

    print("\nOperation completed!")
    print(f"All mask files are now available in: {target_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Target directory for masks: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_masks
Found 25 mask files to move:
  Copied: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional_segmented_dilated.tif to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_masks
  Copied: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Cadherins_regional_segmented_dilated.tif to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_masks
  Copied: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_Cadherins_regional_segmented_dilated.tif to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Holes_masks
  Copied: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_Cadherins_regional_segmented_dilated.tif to /content/drive/MyDrive/knowledge/Universit